In [1]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
import statsmodels.tools.tools as stattools
import jenkspy
from scipy import stats
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [2]:
df = pd.read_csv('E:\\term8\\datamining\\HW\\project\\prepared_data.csv')
breaks = jenkspy.jenks_breaks(df['age'],nb_class = 5)
df['agebin'] = pd.cut(x = df['age'], bins = breaks, right = False, labels = [1,2,3,4,5])

breaks = jenkspy.jenks_breaks(df['bmi'],nb_class = 5)
df['bmibin'] = pd.cut(x = df['bmi'], bins = breaks, right = False, labels = [1,2,3,4,5])

breaks = jenkspy.jenks_breaks(df['avg_glucose_level'],nb_class = 5)
df['glubin'] = pd.cut(x = df['avg_glucose_level'], bins = breaks, right = False, labels = [1,2,3,4,5])
df = df.drop(columns = ['age', 'bmi', 'avg_glucose_level'])
df = pd.get_dummies(df, columns = ['agebin', 'bmibin', 'glubin'])

df_train, df_test = train_test_split(df, test_size = 0.2, random_state = 45)

In [3]:
temp = df_train[df['stroke'] == 1]
temp = temp.sample(n = len(df_train[df_train['stroke'] == 0]) - len(df_train[df_train['stroke'] == 1]),
                   replace = True)#20% of data: 800
df_train = pd.concat([df_train, temp])
y = df_train['stroke'].values
df_train = df_train.drop(columns = ['stroke'])
X = df_train.values
y_test = df_test['stroke'].values
df_test = df_test.drop(columns = ['stroke'])
X_test = df_test.values
%time nb_01 = MultinomialNB().fit(X, y)
%time Y_pred = nb_01.predict(X_test)
print(classification_report(y_test, Y_pred))
print(confusion_matrix(y_test, Y_pred, labels = [0, 1]))

Wall time: 6.03 ms
Wall time: 14.8 ms
              precision    recall  f1-score   support

           0       0.98      0.70      0.81       782
           1       0.16      0.77      0.26        57

    accuracy                           0.70       839
   macro avg       0.57      0.74      0.54       839
weighted avg       0.92      0.70      0.78       839

[[546 236]
 [ 13  44]]


E:\term6\Anaconda\setUp\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [4]:
df = pd.read_csv('E:\\term8\\datamining\\HW\\project\\prepared_data.csv')
df_train, df_test = train_test_split(df, test_size = 0.2, random_state = 45) 
temp = df_train[df['stroke'] == 1]
temp = temp.sample(n = len(df_train[df_train['stroke'] == 0]) - len(df_train[df_train['stroke'] == 1]),
                   replace = True)#20% of data: 800
df_train = pd.concat([df_train, temp])
X = df_train.values[:, 0:20]
y = df_train.values[:, 20]
X_test = df_test.values[:, 0:20]
y_test = df_test.values[:, 20]
%time clf = LogisticRegression(random_state=0).fit(X, y)
%time Y_pred = (clf.predict_proba(X_test)[:,1]>0.5)*1
print(classification_report(y_test, Y_pred))
print(confusion_matrix(y_test, Y_pred, labels = [0, 1]))

E:\term6\Anaconda\setUp\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


Wall time: 30.2 ms
Wall time: 12.1 ms
              precision    recall  f1-score   support

         0.0       0.97      0.73      0.83       782
         1.0       0.16      0.74      0.27        57

    accuracy                           0.73       839
   macro avg       0.57      0.73      0.55       839
weighted avg       0.92      0.73      0.79       839

[[567 215]
 [ 15  42]]


In [5]:
%time clf = SVC(gamma='auto').fit(X, y)
%time Y_pred = clf.predict(X_test)
print(classification_report(y_test, Y_pred))
print(confusion_matrix(y_test, Y_pred, labels = [0, 1]))

Wall time: 1.59 s
Wall time: 404 ms
              precision    recall  f1-score   support

         0.0       0.97      0.70      0.82       782
         1.0       0.15      0.72      0.25        57

    accuracy                           0.70       839
   macro avg       0.56      0.71      0.53       839
weighted avg       0.92      0.70      0.78       839

[[550 232]
 [ 16  41]]


In [6]:
from sklearn.decomposition import PCA
pca_m = PCA(n_components=20).fit(X)
print(np.cumsum(pca_m.explained_variance_ratio_))

[0.27334099 0.44813564 0.56111464 0.64584444 0.72907815 0.78338199
 0.83310977 0.87021602 0.90519734 0.93691883 0.96449501 0.98525364
 0.99989069 0.9999606  1.         1.         1.         1.
 1.         1.        ]


In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
X = df_train.values[:, 0:20]
y = df_train.values[:, 20]
X_test = df_test.values[:, 0:20]
y_test = df_test.values[:, 20]
pca_m = PCA(n_components=14).fit(X)
X = pca_m.transform(X)
X_test = pca_m.transform(X_test)
clf = SVC(gamma='auto')
%time clf.fit(X, y)
%time Y_pred = clf.predict(X_test)
print(classification_report(y_test, Y_pred))
print(confusion_matrix(y_test, Y_pred, labels = [0, 1]))

Wall time: 2.95 s
Wall time: 829 ms
              precision    recall  f1-score   support

         0.0       0.97      0.71      0.82       782
         1.0       0.15      0.70      0.25        57

    accuracy                           0.71       839
   macro avg       0.56      0.71      0.54       839
weighted avg       0.91      0.71      0.78       839

[[558 224]
 [ 17  40]]


In [8]:
df = pd.read_csv('E:\\term8\\datamining\\HW\\project\\prepared_data.csv')
df_train, df_test = train_test_split(df, test_size = 0.2, random_state = 45) 
temp = df_train[df['stroke'] == 1]
temp = temp.sample(n = len(df_train[df_train['stroke'] == 0]) - len(df_train[df_train['stroke'] == 1]),
                   replace = True)#20% of data: 800
df_train = pd.concat([df_train, temp])
X = df_train.values[:, 0:20]
y = df_train.values[:, 20]
X_test = df_test.values[:, 0:20]
y_test = df_test.values[:, 20]
mlp = MLPClassifier(hidden_layer_sizes=(100), max_iter=300,activation = 'relu', solver = 'adam', alpha=1)
%time mlp.fit(X, y)
%time Y_pred = mlp.predict(X_test)
print(classification_report(y_test, Y_pred))
print(confusion_matrix(y_test, Y_pred, labels = [0, 1]))

E:\term6\Anaconda\setUp\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


Wall time: 8.3 s
Wall time: 7.56 ms
              precision    recall  f1-score   support

         0.0       0.97      0.73      0.84       782
         1.0       0.16      0.72      0.27        57

    accuracy                           0.73       839
   macro avg       0.57      0.73      0.55       839
weighted avg       0.92      0.73      0.80       839

[[572 210]
 [ 16  41]]
